In [2]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision.models import vgg19
import torchvision.transforms as transforms
import os.path as osp
from glob import glob
import shutil
from tqdm import tqdm

import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import sys
import json
import random

from torch import optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
#from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import save_image

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
"""
ディレクトリ 構成
root
    | - input
    |        | - annotation
    |        | - cat_face
    |                   | - demo
    |                   | - test
    |                   | - train
    |        | - images
    |                   | - cat
    |
    | - outputs

"""

In [ ]:
ROOT = './'
input_dir = osp.join(ROOT, 'input')
output_dir = osp.join(ROOT, 'output')
os.makedirs(input_dir, exist_ok = True)

# データのダウンロード

In [ ]:
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz -P ./input/
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz -P ./input/

!tar -zxvf ./input/images.tar.gz -C ./input
!tar -zxvf ./input/annotations.tar.gz -C ./input

In [ ]:
!mkdir ./input/images/cat
!mv ./input/images/*.jpg ./input/images/cat/

In [ ]:
# 画像を保存するディレクトリ
image_dir = osp.join(input_dir, 'images/cat')
# ラベルデータを保存するディレクトリ
annotations_dir = osp.join(input_dir, 'annotations')
# ラベルのリストファイルのパス
list_path = osp.join(annotations_dir, 'list.txt')

# データセットのラベル名
cols = ['file_name', 'class_id', 'species', 'breed_id']

#テキストファイルからデータの基礎情報の読み込み
labels = []
with open(list_path, 'r') as f:
    lines = f.read().splitlines()
    for line in lines:
        if line.startswith('#'):
            continue
        labels.append(line.split(' '))
        
labels_df = pd.DataFrame(labels, columns=cols)
labels_df = labels_df[['file_name', 'species']]

cat_label_df = labels_df[labels_df.species=='1']
cat_label_df = cat_label_df.reset_index(drop=True)

In [ ]:
# データセットのディレクトリ
dataset_dir = osp.join(input_dir, 'cat_face')
# データセットの学習データを保存するディレクトリ
train_dir = osp.join(dataset_dir, 'train')
# データセットのテストデータを保存するディレクトリ
test_dir = osp.join(dataset_dir, 'test')
# デモ用のデータを保存するディレクトリ
demo_dir = osp.join(dataset_dir, 'demo')

In [ ]:
seed = 19930124
random_crop_times = 4
# クロップする画像のサイズ
crop_size = (128, 128)
dataset_name = 'cat_face'

In [ ]:
#データクロップ用の関数
def random_crop(image, crop_size):
    h, w, _ = image.shape

    top = np.random.randint(0, h - crop_size[0])
    left = np.random.randint(0, w - crop_size[1])

    bottom = top + crop_size[0]
    right = left + crop_size[1]

    image = image[top:bottom, left:right, :]
    return image

## 猫の名前からテスト用とデモ用を取り出す。

In [ ]:
train_df, test_df = train_test_split(cat_label_df, test_size=5, random_state=seed)
train_df, demo_df = train_test_split(train_df, test_size=1, random_state=seed)

## 上で分離した訓練用のデータに対し、クロップを適用してデータを加工・増量する。

In [ ]:
# 学習に用いる画像
for item in tqdm(train_df.file_name, total=len(train_df)):
    image_name = '{}.jpg'.format(item)
    image_path = osp.join(image_dir, image_name)
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    # 画像のサイズが小さい時は対象から除外する。
    if (h < crop_size[0]) | (w < crop_size[1]):
        print('{} size is invalid. h: {},  w: {}'.format(image_name, h, w))
        continue
    #ランダムクロップ分だけ作る
    for num in range(random_crop_times):
        cropped_image = random_crop(image, crop_size=crop_size)
        image_save_name = '{}_{:03}.jpg'.format(item, num)
        cropped_image_save_path = osp.join(train_dir, image_save_name)
        os.makedirs(osp.dirname(cropped_image_save_path), exist_ok=True)
        cv2.imwrite(cropped_image_save_path, cropped_image)

## テスト用とデモ用の画像を専用のディレクトリに移しておく。

In [ ]:
# 学習の進度確認の画像
for item in test_df.file_name:
    image_name = '{}.jpg'.format(item)
    image_path = osp.join(image_dir, image_name)
    
    image_save_path = osp.join(test_dir, image_name)
    os.makedirs(osp.dirname(image_save_path), exist_ok=True)
    shutil.copy(image_path, image_save_path)
    
# 学習後に超解像を試す画像
for item in demo_df.file_name:
    image_name = '{}.jpg'.format(item)
    image_path = osp.join(image_dir, image_name)
    
    image_save_path = osp.join(demo_dir, image_name)
    os.makedirs(osp.dirname(image_save_path), exist_ok=True)
    shutil.copy(image_path, image_save_path)

In [ ]:
print('学習に用いる画像（最初の数枚）')
plt.figure(figsize=(12, 6))
train_paths = glob(osp.join(train_dir, '*'))
for num, path in enumerate(train_paths[0:5], 1):
    image = cv2.imread(path)
    plt.subplot(1, 5, num)
    plt.imshow(image[:,:,::-1])
plt.show()

print('学習の進度確認の画像')
plt.figure(figsize=(12, 4))
test_paths = glob(osp.join(test_dir, '*'))
for num, path in enumerate(test_paths, 1):
    image = cv2.imread(path)
    plt.subplot(1, 5, num)
    plt.imshow(image[:,:,::-1])
plt.show()

print('学習後に超解像を試す画像')
plt.figure(figsize=(12, 4))
demo_paths = glob(osp.join(demo_dir, '*'))
for num, path in enumerate(demo_paths, 1):
    image = cv2.imread(path)
    plt.subplot(1, 5, num)
    plt.imshow(image[:,:,::-1])
plt.show()

In [ ]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [ ]:
class ImageDataset(Dataset):
    """
    学習のためのDatasetクラス
    32×32の低解像度の本物画像と、128×128の本物画像を出力する
    """
    def __init__(self, dataset_dir, hr_shape):
        hr_height, hr_width = hr_shape
        
        # 低解像度の画像を取得するための処理
        self.lr_transform = transforms.Compose([
            transforms.Resize((hr_height // 4, hr_height // 4), Image.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)])

        # 高像度の画像を取得するための処理
        self.hr_transform = transforms.Compose([
            transforms.Resize((hr_height, hr_height), Image.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)])
        
        self.files = sorted(glob(osp.join(dataset_dir, '*')))
    
    def __getitem__(self, index):
        img = Image.open(self.files[index % len(self.files)])
        img_lr = self.lr_transform(img)
        img_hr = self.hr_transform(img)
        
        return {'lr': img_lr, 'hr': img_hr}
    
    def __len__(self):
        return len(self.files)

In [ ]:
class TestImageDataset(Dataset):
    def __init__(self, dataset_dir):
        self.hr_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean, std)])
        self.files = sorted(glob(osp.join(dataset_dir, '*')))
    
    def lr_transform(self, img, img_size):
        img_width, img_height = img_size
        self.__lr_transform = transforms.Compose([
            transforms.Resize((img_height // 4, 
                               img_width // 4), 
                               Image.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)])
        img = self.__lr_transform(img)
        return img
            
    def __getitem__(self, index):
        img = Image.open(self.files[index % len(self.files)])
        img_size = img.size
        img_lr = self.lr_transform(img, img_size)
        img_hr = self.hr_transform(img)        
        return {'lr': img_lr, 'hr': img_hr}
    
    def __len__(self):
        return len(self.files)


# parts of ESRGAN

## 生成器の準備

## Dense Residual Blockの構築

In [ ]:

class DenseResidualBlock(nn.Module):
    def __init__(self, filters, res_scale = 0.2):
        super().__init__()
        self.res_scale = res_scale
        
        def block(in_features, non_linearity = True):
            layers = [nn.Conv2d(in_features, filters, 3, 1, 1, bias = True)]
            if non_linearity:
                layers += [nn.LeakyReLU()]
            return nn.Sequential(*layers)
    
        self.b1 = block(in_features = 1 * filters)
        self.b2 = block(in_features = 2 * filters)
        self.b3 = block(in_features = 3 * filters)
        self.b4 = block(in_features = 4 * filters)
        self.b5 = block(in_features = 5 * filters, non_linearity = False)
        self.blocks = [self.b1, self.b2, self.b3, self.b4, self.b5]
    
    def forward(self, x):
        inputs = x
        for block in self.blocks:
            out = block(inputs)
            inputs = torch.cat([inputs, out], dim = 1)
        return out.mul(self.res_scale) + x

In [ ]:
# サイズの確認
x = torch.randn(10, 64, 5, 5)
D = DenseResidualBlock(filters = 64)
D(x).shape

## Residual in Residual Blockの構築

In [ ]:
# residual-blockの再帰
class ResidualInResidualDenseBlock(nn.Module):
    def __init__(self, filters, res_scale=0.2):
        super().__init__()
        self.res_scale = res_scale
        self.dense_blocks = nn.Sequential(
            DenseResidualBlock(filters), 
            DenseResidualBlock(filters), 
            DenseResidualBlock(filters)
        )
    
    def forward(self, x):
        return self.dense_blocks(x).mul(self.res_scale) + x

In [ ]:
# サイズの確認
DD = ResidualInResidualDenseBlock(filters = 64)
DD(x).shape

## 構築してきたパーツを組み合わせて生成器を構築

In [ ]:
"""
Conv2D
RinRD-Block
     .
     .
RinRD-Block
Conv2D
|---------------|
|Conve2D     |
|LeaklyReLU |
|Pixelshuffle |
|---------------|
     .
     .
|---------------|
|Conve2D     |
|LeaklyReLU |
|Pixelshuffle |
|---------------|
Conv2D
LeaklyReLU
Conv2D
"""

class GeneratorRRDB(nn.Module):
    def __init__(self, channels, filters = 64, num_res_blocks = 16, num_upsample = 2):
        super().__init__()
        
        self.conv1 = nn.Conv2d(channels, filters, kernel_size = 3, stride = 1, padding = 1)
        self.res_blocks = nn.Sequential(*[ResidualInResidualDenseBlock(filters)  for _ in range(num_res_blocks)])
        self.conv2 = nn.Conv2d(filters, filters, kernel_size = 3, stride = 1, padding = 1)
        
        upsample_layers = []
        
        for _ in range(num_upsample):
            upsample_layers += [
                nn.Conv2d(filters, filters * 4, kernel_size = 3, stride = 1, padding = 1),
                nn.LeakyReLU(),
                nn.PixelShuffle(upscale_factor = 2),
            ]
        self.upsampling = nn.Sequential(*upsample_layers)
        self.conv3 = nn.Sequential(
            nn.Conv2d(filters, filters, kernel_size = 3, stride = 1, padding = 1),
            nn.LeakyReLU(),
            nn.Conv2d(filters, channels, kernel_size = 3, stride = 1, padding = 1),
        )
    
    def forward(self, x):
        out1 = self.conv1(x)
        #print(1,out1.shape)
        out = self.res_blocks(out1)
        #print(2,out.shape)
        out2 = self.conv2(out)
        #print(3,out2.shape)
        out = torch.add(out1, out2)
        #print(4,out.shape)
        out = self.upsampling(out)
        #print(5,out.shape)
        out = self.conv3(out)
        #print(6,out.shape)
        return out

In [ ]:
# サイズの確認
# ex : (10,3,32,32) --> (10, 3, 128, 128)
y = torch.randn(10,3,32,32)
DDD = GeneratorRRDB(3, filters = 64, num_res_blocks = 23)
DDD(y).shape

## 特徴量抽出層

In [ ]:
class FeatureExtractor(nn.Module):
    """
    vgg19を応用した特徴量抽出器
    """
    def __init__(self):
        super().__init__()
        vgg19_model = vgg19(pretrained = True)
        self.vgg19_54 = nn.Sequential(*list(vgg19_model.features.children())[:35])

    def forward(self, x):
        return self.vgg19_54(x)

In [ ]:
# ex : (10, 3, 128, 128) --> (10, 512, 8,8)
y = torch.randn(10,3,32,32)
FE = FeatureExtractor()
FE(y).shape

In [5]:
y = torch.randn(10,3,32,32)
(y-y.mean(0, keepdim=True)).shape

torch.Size([10, 3, 32, 32])

## 識別器の準備

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super().__init__()
                
        self.input_shape = input_shape
        in_channels, in_height, in_width = self.input_shape
        patch_h, patch_w = int(in_height / 2 ** 4), int(in_width / 2 ** 4)
        self.output_shape = (1, patch_h, patch_w)
    
        def discriminator_block(in_filters, out_filters, first_block = False):
            layers = []
            layers.append(nn.Conv2d(in_filters, 
                                    out_filters, 
                                    kernel_size = 3, 
                                    stride = 1, 
                                    padding = 1))
            if not first_block:
                layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace = True))
            layers.append(nn.Conv2d(out_filters, 
                                    out_filters, 
                                    kernel_size = 3, 
                                    stride = 2, 
                                    padding = 1))
            layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace = True))
            return layers
        
        layers = []
        in_filters = in_channels
        for i, out_filters in enumerate([64, 128, 256, 512]):
            #print(discriminator_block(in_filters, out_filters,  first_block=(i == 0)))
            layers.extend(discriminator_block(in_filters, out_filters, first_block=(i == 0)))
            in_filters = out_filters
        
        layers.append(nn.Conv2d(out_filters,  1, kernel_size = 3, stride = 1,  padding = 1))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, img):
        return self.model(img)

## 構築してきたパーツを組み合わせてESRGANを構築

In [ ]:
class ESRGAN():
    def __init__(self, opt):
        # 生成器・識別器の設定
        self.generator = GeneratorRRDB(opt.channels, filters = 64, num_res_blocks = opt.residual_blocks).to(opt.device)
        self.discriminator = Discriminator(input_shape = (opt.channels, *hr_shape)).to(opt.device)

        # 特徴量抽出器の設定
        self.feature_extractor = FeatureExtractor().to(opt.device)
        self.feature_extractor.eval()
        
        # 損失関数の設定
        self.criterion_GAN = nn.BCEWithLogitsLoss().to(opt.device)
        self.criterion_content = nn.L1Loss().to(opt.device)
        self.criterion_pixel = nn.L1Loss().to(opt.device)

        # オプティマイザーの設定
        self.optimizer_G = optim.Adam(self.generator.parameters(), lr = opt.lr, betas = (opt.b1, opt.b2))
        self.optimizer_D = optim.Adam(self.discriminator.parameters(), lr = opt.lr, betas = (opt.b1, opt.b2))

        # デバイスとテンソル型の定義
        self.Tensor = torch.Tensor
        self.dev = opt.device
        
    #==================================================================================
    
    def pre_train(self, imgs, batch_num):
        imgs_lr = imgs['lr'].type(torch.Tensor).to(self.dev)
        imgs_hr = imgs['hr'].type(torch.Tensor).to(self.dev)

        valid = torch.tensor(np.ones((imgs_lr.size(0), *self.discriminator.output_shape)), requires_grad=False).to(self.dev)
        fake = torch.tensor(np.zeros((imgs_lr.size(0), *self.discriminator.output_shape)), requires_grad=False).to(self.dev)

        # 勾配初期化
        self.optimizer_G.zero_grad()

        # 低解像度 --> 高解像度を実行し、ピクセル単位の損失計算
        gen_hr = self.generator(imgs_lr)
        loss_pixel = self.criterion_pixel(gen_hr, imgs_hr)

        # 画素単位の損失であるloss_pixelで事前学習を行う
        loss_pixel.backward()
        self.optimizer_G.step()
        train_info = {'epoch': epoch, 'batch_num': batch_num, 'loss_pixel': loss_pixel.item()}
        """
        if batch_num == 1:
            sys.stdout.write('\n{}'.format(train_info))
        else:
            sys.stdout.write('\r{}'.format('\t'*20))
            sys.stdout.write('\r{}'.format(train_info))
        """
        
        sys.stdout.write('\r{}'.format('\t'*20))
        sys.stdout.write('\r{}'.format(train_info))
        sys.stdout.flush()
        
    #============================================================================
    
    def train(self, imgs, batch_num):
        imgs_lr = imgs['lr'].type(self.Tensor).to(self.dev)
        imgs_hr = imgs['hr'].type(self.Tensor).to(self.dev)

        ################### 
        #####生成器の損失#####
        ###################
        # 正解ラベル
        valid = torch.tensor(np.ones((imgs_lr.size(0), *self.discriminator.output_shape)), requires_grad = False).to(self.dev)
        fake = torch.tensor(np.zeros((imgs_lr.size(0), *self.discriminator.output_shape)),requires_grad = False).to(self.dev)

        # 低解像度 --> 高解像度
        self.optimizer_G.zero_grad()
        gen_hr = self.generator(imgs_lr)

        # (1)ピクセル単位の損失計算
        loss_pixel = self.criterion_pixel(gen_hr, imgs_hr)

        # (2)Adversarial loss
        # 本物画像と超解像画像の識別器による判定を見る
        pred_real = self.discriminator(imgs_hr).detach()
        pred_fake = self.discriminator(gen_hr)
        loss_GAN = self.criterion_GAN(pred_fake - pred_real.mean(0, keepdim=True), valid)

        # (3)Perceptual loss
        #特徴量の比較、シンプルな損失関数
        gen_feature = self.feature_extractor(gen_hr)
        real_feature = self.feature_extractor(imgs_hr).detach()
        loss_content = self.criterion_content(gen_feature, real_feature)

        
        loss_G = loss_content + opt.lambda_adv * loss_GAN + opt.lambda_pixel * loss_pixel
        loss_G.backward()
        self.optimizer_G.step()

        
        ################### 
        #####識別機の損失#####
        ###################
        self.optimizer_D.zero_grad()
        pred_real = self.discriminator(imgs_hr)
        pred_fake = self.discriminator(gen_hr.detach())

        loss_real = self.criterion_GAN(pred_real - pred_fake.mean(0, keepdim=True), valid)            
        loss_fake = self.criterion_GAN(pred_fake - pred_real.mean(0, keepdim=True), fake)    
        loss_D = (loss_real + loss_fake) / 2
        loss_D.backward()
        self.optimizer_D.step()

        train_info = {'epoch': epoch, 'batch_num': batch_num,  'loss_D': loss_D.item(), 'loss_G': loss_G.item(),
                      'loss_content': loss_content.item(), 'loss_GAN': loss_GAN.item(), 'loss_pixel': loss_pixel.item(),}
        """
        if batch_num == 1:
            sys.stdout.write('\n{}'.format(train_info))
        else:
            sys.stdout.write('\r{}'.format('\t'*20))
            sys.stdout.write('\r{}'.format(train_info))
        """
        sys.stdout.write('\r{}'.format('\t'*20))
        sys.stdout.write('\r{}'.format(train_info))
        sys.stdout.flush()



    def save_image(self, imgs, batches_done):
        """
        画像の保存
        """
        with torch.no_grad():
            imgs_lr = imgs["lr"].type(self.Tensor).to(self.dev)
            gen_hr = self.generator(imgs_lr)
            gen_hr = denormalize(gen_hr)

            image_batch_save_dir = osp.join(image_test_save_dir, '{:03}'.format(i))
            gen_hr_dir = osp.join(image_batch_save_dir, "hr_image")
            os.makedirs(image_batch_save_dir, exist_ok=True)
            save_image(gen_hr, osp.join(image_batch_save_dir, "{:09}.png".format(batches_done)), nrow=1, normalize=False)

    def save_weight(self, batches_done):
        """
        重みの保存
        """
        generator_weight_path = osp.join(weight_save_dir, "generator_{:08}.pth".format(batches_done))
        discriminator_weight_path = osp.join(weight_save_dir, "discriminator_{:08}.pth".format(batches_done))

        torch.save(self.generator.state_dict(), generator_weight_path)
        torch.save(self.discriminator.state_dict(), discriminator_weight_path)

In [ ]:
def denormalize(tensors):
    """
    平均と標準偏差を使ってデータ値の加工
    """
    for c in range(3):
        tensors[:, c].mul_(std[c]).add_(mean[c])
    return torch.clamp(tensors, 0, 255)

In [ ]:
image_train_save_dir = osp.join(output_dir, 'image', 'train')
image_test_save_dir = osp.join(output_dir, 'image', 'test')
weight_save_dir = osp.join(output_dir, 'weight')
param_save_path = osp.join(output_dir, 'param.json')

save_dirs = [image_train_save_dir, image_test_save_dir, weight_save_dir]
for save_dir in save_dirs:
    print(save_dir)
    os.makedirs(save_dir, exist_ok=True)

In [ ]:
def save_json(file, save_path, mode):
    """Jsonファイルを保存
    """
    with open(save_path, mode) as outfile:
        json.dump(file, outfile, indent=4)

In [ ]:
class Opts():
    def __init__(self):
        self.n_epoch = 50
        self.residual_blocks = 23
        self.lr = 0.0002
        self.b1 = 0.9
        self.b2 = 0.999
        self.batch_size = 16
        self.n_cpu = 8
        self.warmup_batches = 5#500
        self.lambda_adv = 5e-3
        self.lambda_pixel = 1e-2
        self.pretrained = False
        self.dataset_name = 'cat'
        self.sample_interval = 100
        self.checkpoint_interval = 1000
        self.hr_height = 128
        self.hr_width = 128
        self.channels = 3
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        print(self.device)
    def to_dict(self):
        parameters = {
            'n_epoch': self.n_epoch,
            'hr_height': self.hr_height,
            'residual_blocks': self.residual_blocks,
            'lr': self.lr,
            'b1': self.b1,
            'b2': self.b2,
            'batch_size': self.batch_size,
            'n_cpu': self.n_cpu,
            'warmup_batches': self.warmup_batches,
            'lambda_adv': self.lambda_adv,
            'lambda_pixel': self.lambda_pixel,
            'pretrained': self.pretrained,
            'dataset_name': self.dataset_name,
            'sample_interval': self.sample_interval,
            'checkpoint_interval': self.checkpoint_interval,
            'hr_height': self.hr_height,
            'hr_width': self.hr_width,
            'channels': self.channels,
            'device': str(self.device),
        }
        return parameters
opt = Opts()
save_json(opt.to_dict(), param_save_path, 'w')

In [ ]:
hr_shape = (opt.hr_height, opt.hr_height)

In [ ]:
train_dataloader = DataLoader(
    ImageDataset(train_dir, hr_shape = hr_shape),
    batch_size = opt.batch_size,
    shuffle = True,
    num_workers = opt.n_cpu,
)

test_dataloader = DataLoader(
    TestImageDataset(test_dir),
    batch_size = 1,
    shuffle = False,
    num_workers = opt.n_cpu,
)

In [ ]:
esrgan = ESRGAN(opt)

In [ ]:
for epoch in range(1, opt.n_epoch + 1):
    print("\ncurrent epoch : ", epoch)
    for batch_num, imgs in enumerate(train_dataloader):
        batches_done = (epoch - 1) * len(train_dataloader) + batch_num
        # 事前学習
        if batches_done <= opt.warmup_batches:#500
            esrgan.pre_train(imgs, batch_num)
        # メイン学習
        else:
            esrgan.train(imgs, batch_num)
        # 高解像度の生成画像の保存
        if batches_done % opt.sample_interval == 0:
            for i, imgs in enumerate(test_dataloader):
                esrgan.save_image(imgs, batches_done)
        # 学習した重みの保存
        if batches_done % opt.checkpoint_interval == 0:
            esrgan.save_weight(batches_done)